# Lab 2: Superando o Ruído

Neste laboratório, mergulhamos no mundo do ruído quântico e exploramos as diferentes técnicas para navegar pelos desafios do ruído, a fim de obter bons resultados com os computadores quânticos atuais. No contexto de um problema Max-cut de pequeno porte, fornecemos um guia passo a passo para reduzir o ruído e executar um algoritmo quântico em um computador quântico. Apresentamos diferentes técnicas de transpilação e mitigação de erros para minimizar o erro e garantir que os resultados permaneçam corretos apesar do ruído. Finalmente, o laboratório conclui com um exercício bônus onde todas as técnicas discutidas são implementadas em hardware quântico real.

# Sumário

0. [Requisitos](#requirements)
1. [Introdução](#intro)  
    - [Espírito do Laboratório](#spirit)
    - [O que é ruído quântico?](#quantum-noise)  
    - [Fontes de ruído](#sources)
        * [Exercício 1: Encontre os qubits com T1, T2 mais longos, maiores fidelidades de porta e menores erros de leitura](#exercise_1)
2. [Problema Max-cut](#max-cut)
    - [Exercício 2: Do Grafo ao Hamiltoniano](#exercise_2)
3. [Simulador quântico com ruído](#noisy-simulator)
    - [Escolhendo o backend](#choosing-backend)
    - [Exercício 3: Contagem de Erros](#exercise_3)
    - [Estimando erros usando NEAT](#neat)
4. [Transpilação](#transpilation)
    - [Exercício 4: Bom Mapeamento](#exercise_4)
    - [Exercício 5: Melhor Mapeamento](#exercise_5)
5. [Mitigação de Erros (EM)](#em)
    - [Extrapolação de Ruído Zero (ZNE)](#zne)
    - [Exercício 6: Implementando ZNE](#exercise_6)
6. [Conclusões](#conclusions)
7. [Desafio Bônus: Ampliando!](#bonus)
8. [Referências](#references)

<a id="requirements"></a>

## 0. Requisitos 

Para executar este laboratório, você precisará das seguintes dependências do Python:

In [ ]:
%pip install -U qiskit "qc-grader[qiskit,jupyter] @ git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git"
%pip install 'qiskit[visualization]'
%pip install qiskit-ibm-runtime
%pip install qiskit-aer
%pip install rustworkx
%pip install pylatexenc
%pip install qiskit matplotlib
%pip install qiskit-aer

In [ ]:
import qiskit
import qc_grader

print(f"Qiskit version: {qiskit.__version__}")
print(f"Grader version: {qc_grader.__version__}")

## 1. Introdução <a id="intro"></a>

### 1.1 Espírito do laboratório <a id="spirit"></a>

O ruído é um tópico central no projeto e uso de computadores quânticos, e é uma das principais características dos dispositivos atuais. Mas o que exatamente significa ruído na informação quântica? Geralmente nos referimos ao termo ruído como qualquer transformação indesejada que perturba o resultado esperado da medição de um estado quântico. Categorizamos os erros em dois grupos:

In [ ]:
# Verifica se a conta foi salva corretamente
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService(name="qgss-2025")
service.saved_accounts()

### 1.2 O que é ruído quântico? <a id="quantum-noise"></a>

- **Erros incoerentes**: A origem desses erros é naturalmente quântica, pois vem da interação do sistema quântico com o ambiente. Erros incoerentes são a razão pela qual a maioria dos computadores quânticos atuais precisa ser resfriada a uma temperatura da ordem de mK, para reduzir essas interações indesejadas e preservar a coerência do sistema. Essas interações criam estados mistos, que introduzem aleatoriedade na saída e tornam esses erros particularmente problemáticos. Um exemplo concreto é o tempo de relaxamento $T_1$, que descreve como um qubit no estado excitado $|1\rangle$ decai espontaneamente para o estado fundamental $|0\rangle$ ao longo do tempo. Se uma medição for atrasada, esse decaimento pode fazer com que o qubit seja lido incorretamente como $|0\rangle$ quando deveria ser $|1\rangle$.

- **Erros coerentes**: Esses erros são determinísticos e resultam de imperfeições na implementação de portas quânticas ou na calibração do sistema. Por exemplo, se uma porta RX (rotação em torno do eixo X) deveria girar um qubit em $\pi/2$ radianos, mas na verdade o gira em $\pi/2 + \epsilon$ devido a imprecisões no campo de controle, isso introduziria um erro coerente sistemático. Embora esses erros sejam problemáticos, eles são mais previsíveis e podem ser corrigidos com técnicas de calibração adequadas.

In [ ]:
import rustworkx as rx
import numpy as np
import matplotlib.pyplot as plt
from rustworkx.visualization import mpl_draw as draw_graph
from qiskit_ibm_runtime import QiskitRuntimeService
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import SparsePauliOp, Statevector, DensityMatrix, Operator
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.transpiler import Layout

from qiskit_ibm_runtime import (
    Session,
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    EstimatorOptions,
)
from qiskit_ibm_runtime.debug_tools import Neat
from qiskit_aer import AerSimulator

from utils import zne_method, plot_zne, plot_backend_errors_and_counts
from qc_grader.challenges.qgss_2025 import (
    grade_lab2_ex1,
    grade_lab2_ex2,
    grade_lab2_ex3,
    grade_lab2_ex4,
    grade_lab2_ex5,
    grade_lab2_ex6a,
    grade_lab2_ex6b,
)

# Define seed para reprodutibilidade
seed = 43

### 1.3 Fontes de ruído <a id="sources"></a>

Como mencionado acima, algumas fontes de ruído vêm de diferentes fatores, incluindo a implementação imperfeita de portas quânticas como pulsos eletromagnéticos, erros de leitura, decoerência do sistema quântico, e assim por diante.

Entre as diferentes métricas que podem ser usadas para avaliar quão bom ou resiliente ao ruído um computador quântico (ou um qubit) é contra as fontes de ruído, podemos distinguir algumas:

<a id="exercise_1"></a>

<b>Exercício 1: Encontre os melhores qubits</b> 

Neste primeiro exercício, você fornecerá o melhor valor de cada uma das seguintes métricas, bem como o qubit, ou par de qubits, que possui esse valor:

- **T1 (tempo de relaxamento)**: Qubit com o maior T1 e seu valor
- **T2 (tempo de coerência)**: Qubit com o maior T2 e seu valor  
- **Fidelidade de porta de um qubit**: Qubit com a maior fidelidade de porta de um qubit e seu valor
- **Fidelidade de porta de dois qubits**: Par de qubits com a maior fidelidade de porta CNOT e seu valor
- **Erro de leitura**: Qubit com o menor erro de leitura e seu valor

In [ ]:
# Execute para criar arrays de propriedades
service = QiskitRuntimeService(name="qgss-2025")
# Definimos um backend específico
brisbane_backend = service.backend("ibm_brisbane")
# Obtemos as propriedades do sistema, número de qubits e mapa de acoplamento
properties = brisbane_backend.properties()
num_qubits = brisbane_backend.num_qubits
coupling_map = brisbane_backend.coupling_map

# Definimos várias listas de métricas para todos os qubits do backend
t1, t2, gate_error_x, readout_error, gate_error_ecr = [], [], [], [], []
for i in range(num_qubits):
    t1.append(properties.t1(i))
    t2.append(properties.t2(i))
    gate_error_x.append(properties.gate_error(gate="x", qubits=i))
    readout_error.append(properties.readout_error(i))
for pair in coupling_map:
    gate_error_ecr.append(properties.gate_error(gate="ecr", qubits=pair))

In [ ]:
def find_best_metrics(backend: QiskitRuntimeService.backend) -> list[tuple[int or list, float]]:
    """Encontra os melhores qubits e par de qubits baseado em várias métricas de hardware."""
    properties = backend.properties()
    num_qubits = backend.num_qubits
    coupling_map = backend.coupling_map

    # Define listas de métricas para o backend
    t1, t2, gate_error_x, readout_error, gate_error_ecr = [], [], [], [], []
    for i in range(num_qubits):
        t1.append(properties.t1(i))
        t2.append(properties.t2(i))
        gate_error_x.append(properties.gate_error(gate="x", qubits=i))
        readout_error.append(properties.readout_error(i))
    for pair in coupling_map:
        gate_error_ecr.append(properties.gate_error(gate="ecr", qubits=pair))

    # Objetivo: Obter o melhor valor e o índice ou índices dos qubits das seguintes métricas:
    # encontrar o melhor qubit (index_t1_max) com o maior T1 e seu valor (max_t1)
    index_t1_max = t1.index(max(t1))
    max_t1 = max(t1)

    # encontrar o melhor qubit (index_t2_max) com o maior T2 e seu valor (max_t2)
    index_t2_max = t2.index(max(t2))
    max_t2 = max(t2)

    # encontrar o melhor qubit (index_min_x_error) com o menor erro de porta x e seu valor (min_x_error)
    index_min_x_error = gate_error_x.index(min(gate_error_x))
    min_x_error = min(gate_error_x)

    # encontrar o melhor qubit (index_min_readout) com o menor erro de leitura e seu valor (min_readout)
    index_min_readout = readout_error.index(min(readout_error))
    min_readout = min(readout_error)

    # encontrar o melhor par de qubits com erro ECR mínimo (min_ecr_pair) e seu valor (min_ecr_error)
    min_ecr_error = min(gate_error_ecr)
    coupling_pairs = list(coupling_map)
    min_ecr_pair = list(coupling_pairs[gate_error_ecr.index(min_ecr_error)])

    solutions = [
        [int(index_t1_max), max_t1],
        [int(index_t2_max), max_t2],
        [int(index_min_x_error), min_x_error],
        [int(index_min_readout), min_readout],
        [min_ecr_pair, min_ecr_error]
    ]
    return solutions

In [ ]:
# Submeta sua resposta usando o código a seguir
grade_lab2_ex1(find_best_metrics)

In [ ]:
# Função de teste para ver a saída
from qiskit_ibm_runtime.fake_provider import FakeBrisbane
test_backend = FakeBrisbane()
result = find_best_metrics(test_backend)
print("Saída da função:", result)
print("Tamanho do resultado:", len(result))
for i, item in enumerate(result):
    print(f"Item {i}: {item}, tipo: {type(item[0])}, {type(item[1])}")

# 2. O problema: Max-cut <a id="max-cut"></a>

O problema do corte máximo ([Max-cut](https://en.wikipedia.org/wiki/Maximum_cut)) é um problema de grafo que se enquadra na categoria de [NP-difícil](https://en.wikipedia.org/wiki/NP-hardnes), o que significa que não existe um algoritmo que possa resolvê-lo em tempo polinomial. O Max-cut é um problema de otimização com uma ampla gama de aplicações, incluindo clustering, ciência de redes, física estatística e aprendizado de máquina. O objetivo deste problema é dividir os nós de um grafo em dois conjuntos usando um único corte, de forma que o número de arestas que conectam os dois conjuntos seja maximizado.

![Ilustração de um problema de max-cut](attachment:image.png)

Neste laboratório, resolveremos um problema de Max-cut usando um algoritmo quântico. Também estudaremos como o ruído quântico afeta nossa solução e discutiremos estratégias para reduzir seu impacto, garantindo que ainda possamos obter resultados precisos apesar do ruído.

Agora que temos um pouco de contexto sobre o problema Max-cut, vamos escolher um grafo específico para o qual queremos encontrar o corte máximo. Em particular, escolheremos o grafo do mapa de acoplamento de um computador quântico hipotético com conectividade total $^*$.

<a id="exercise_2"></a>

<b>Exercício 2: Do Grafo ao Hamiltoniano</b> 

Neste segundo exercício, você deve encontrar uma maneira de mapear o problema do grafo dado para um Hamiltoniano usando as portas de identidade e Pauli.

Para o problema Max-cut em um grafo $G = (V, E)$, queremos particionar os vértices $V$ em dois conjuntos. O objetivo é maximizar o número de arestas que conectam vértices em conjuntos diferentes. Podemos expressar esse objetivo usando nossas variáveis de spin **s<sub>i</sub>**. Se dois nós conectados `i` e `j` estão em partições diferentes (s<sub>i</sub> ≠ s<sub>j</sub>), o termo $s_i * s_j$ será -1. Se eles estiverem na mesma partição (s<sub>i</sub> = s<sub>j</sub>), o termo será +1.

$$C = -\frac{1}{2} \sum_{(i,j) \in E} (1 - s_i s_j)$$

Aqui, substituímos as variáveis de spin clássicas `s_i` por suas contrapartes quânticas, os operadores **Z<sub>i</sub>** de Pauli, que atuam no `i`-ésimo qubit. O estado fundamental deste Hamiltoniano é o arranjo específico de estados de qubit (`|0⟩` ou `|1⟩`) que minimiza essa energia, nos dando diretamente a solução para o problema Max-cut.

$$H = -\frac{1}{2} \sum_{(i,j) \in E} (I - Z_i Z_j)$$

**Seu Objetivo:** Converter o objeto 'grafo' que acabamos de criar em um Hamiltoniano de Ising, que é a função de custo para o nosso problema Max-cut.

In [ ]:
# Definimos o seed
seed = 43
# Definimos o número de nós:
n = 5
# Definimos o grafo
graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
generic_backend = GenericBackendV2(n, seed=seed)
weights = 1
# Tornamos explicitamente assimétrico para ter um conjunto menor de soluções
graph.add_edges_from([(edge[0], edge[1], weights) for edge in generic_backend.coupling_map][:-1])
draw_graph(graph, node_size=200, with_labels=True, width=1)

# 2.3 Solução QAOA <a id="qaoa-solution"></a>

Agora que mapeamos com sucesso nosso problema Max-cut para um Hamiltoniano de Ising, traduzimos um problema de grafo clássico para um problema de busca de estado fundamental quântico. A solução para o problema original está codificada no estado com a menor energia possível (o "estado fundamental") deste Hamiltoniano.

Vários algoritmos quânticos podem ser empregados para encontrar este estado fundamental. Um dos mais proeminentes é o Algoritmo de Otimização Aproximada Quântica ([QAOA](https://en.wikipedia.org/wiki/Quantum_optimization_algorithms)). O QAOA é conhecido por sua adaptabilidade, profundidade de circuito relativamente baixa e forte desempenho em uma variedade de tarefas de otimização.

Se você quiser uma descrição completa do QAOA, recomendamos este [tutorial](https://quantum.cloud.ibm.com/docs/tutorials/quantum-approximate-optimization-algorithm), bem como a lição [QAOA em escala de utilidade](https://quantum.cloud.ibm.com/learning/courses/quantum-computing-in-practice/utility-scale-qaoa) do curso de computação quântica na prática. Aqui, comentaremos brevemente sobre a ideia principal por trás dele.

O QAOA é um algoritmo quântico variacional inspirado no teorema adiabático, que afirma que um sistema quântico permanece em seu estado fundamental se evoluir lentamente o suficiente. O QAOA imita isso começando no estado fundamental de um Hamiltoniano conhecido (o misturador) e evoluindo em direção ao estado fundamental de um Hamiltoniano que codifica nosso problema (o custo). Isso é feito através de camadas alternadas dos dois Hamiltonianos, com cada passo controlado por parâmetros ajustáveis que guiam o sistema em direção à solução ótima.

Após esta pequena explicação da teoria, vamos praticar usando o `QAOAAnsatz` do Qiskit para implementar o QAOA e resolver nosso problema Max-cut.

Depois de definirmos o circuito QAOA, devemos passá-lo para o gerenciador de passes para transpilá-lo para as portas nativas do backend.

In [ ]:
def graph_to_Pauli(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Converte o grafo para lista de Pauli."""
    pauli_list = []
    n = graph.num_nodes()
    
    # Usa edge_list() como no tutorial da IBM
    for edge in graph.edge_list():
        pauli_word = ['I'] * n
        pauli_word[edge[0]] = 'Z'
        pauli_word[edge[1]] = 'Z'
        
        # Obtém o peso usando get_edge_data como no tutorial
        weight = graph.get_edge_data(edge[0], edge[1])
        
        # Inverte a string de Pauli como no tutorial da IBM
        pauli_list.append((''.join(pauli_word)[::-1], weight))
    
    return pauli_list

In [ ]:
# Submeta sua resposta usando o código a seguir
grade_lab2_ex2(graph_to_Pauli)

In [ ]:
# Debug simples para pesos das arestas do grafo
print("Propriedades do grafo:")
print(f"Número de nós: {graph.num_nodes()}")
print(f"Número de arestas: {graph.num_edges()}")
print(f"Lista de arestas primeiras 5: {graph.edge_list()[:5]}")
print(f"Lista de arestas ponderadas primeiras 5: {graph.weighted_edge_list()[:5]}")

# Verifica se os pesos importam
edge_weights = set()
for _, _, weight in graph.weighted_edge_list():
    edge_weights.add(weight)
print(f"Pesos únicos das arestas: {edge_weights}")

# Testa abordagem simples
simple_paulis = []
for i, j, weight in graph.weighted_edge_list():
    pauli_word = ['I'] * graph.num_nodes()
    pauli_word[i] = 'Z'
    pauli_word[j] = 'Z'
    simple_paulis.append((''.join(pauli_word), weight))

print(f"\nPrimeiros 3 termos de Pauli com pesos:")
for i, (pauli, coeff) in enumerate(simple_paulis[:3]):
    print(f"{pauli}: {coeff}")

No próximo exercício, você contará o erro total acumulado da aplicação dos diferentes circuitos quânticos aos três backends. Podemos fazer isso usando [`backend.properties()`](https://quantum.cloud.ibm.com/docs/en/guides/get-qpu-information#dynamic-backend-information), [`circuit.data`](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.QuantumCircuit) e [`backend.configuration()`](https://quantum.cloud.ibm.com/docs/api/qiskit/qiskit.circuit.QuantumCircuit), pois eles nos fornecerão as informações necessárias.

<a id="exercise_3"></a>

<b>Exercício 3: Contagem de Erros</b> 

Neste terceiro exercício, você estimará o erro total introduzido por todas as instruções ao executar circuitos quânticos em diferentes backends, completando a função `accululated_errors`.<br> 

**Dica**: Você pode estimar o erro total acumulado simplesmente somando todos os erros de porta individuais. Por exemplo, se um circuito contém apenas uma porta RZ e uma CNOT, o erro total seria aproximadamente RZ_error + CNOT_error.

Embora essa aproximação possa produzir resultados semelhantes, o objetivo deste exercício é ensiná-lo a acessar informações detalhadas do backend e realizar uma estimativa mais precisa das taxas de erro específicas por qubit.

In [ ]:
layers = 2
qaoa_circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=layers)
qaoa_circuit.measure_all()
qaoa_circuit.draw("mpl")

Após definirmos o circuito QAOA, devemos passá-lo para o gerenciador de passes para transpilá-lo para as portas nativas do backend.

In [ ]:
# Cria o Hamiltoniano de custo a partir dos termos de Pauli
max_cut_paulis = graph_to_Pauli(graph)
cost_hamiltonian = SparsePauliOp.from_list(max_cut_paulis)

In [ ]:
# Cria o gerenciador de passes para transpilação

pm = generate_preset_pass_manager(
    optimization_level=3, backend=generic_backend, seed_transpiler=seed
)

qaoa_circuit_transpiled = pm.run(qaoa_circuit)
qaoa_circuit_transpiled.draw("mpl", fold=False, idle_wires=False)

Vamos ver quais backends estão disponíveis para você.

In [ ]:
init_params = np.zeros(2 * layers)

Vamos plotar os resultados.

In [ ]:
objective_func_vals = []


def cost_func_estimator(
    params: list, ansatz: QuantumCircuit, isa_hamiltonian: SparsePauliOp, estimator: Estimator
) -> float:
    """Compute the cost function value using a parameterized ansatz and an estimator for a given Hamiltonian."""
    if isa_hamiltonian.num_qubits != ansatz.num_qubits:
        isa_hamiltonian = isa_hamiltonian.apply_layout(ansatz.layout)
    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])
    results = job.result()[0]
    cost = results.data.evs
    objective_func_vals.append(cost)
    return cost


def train_qaoa(
    params: list,
    circuit: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    backend: QiskitRuntimeService.backend,
) -> tuple:
    """Optimize QAOA parameters using COBYLA and an estimator on a given backend."""
    with Session(backend=backend) as session:
        options = {"simulator": {"seed_simulator": seed}}
        estimator = Estimator(mode=session, options=options)
        estimator.options.default_shots = 100000

        result = minimize(
            cost_func_estimator,
            params,
            args=(circuit, hamiltonian, estimator),
            method="COBYLA",
            options={"maxiter": 200, "rhobeg": 1, "catol": 1e-3, "tol": 0.0001},
        )
    print(result)
    return result, objective_func_vals


result_qaoa, objective_func_vals = train_qaoa(
    init_params, qaoa_circuit_transpiled, cost_hamiltonian, generic_backend
)

Como esperado, `ibm_torino` nos fornece a maior probabilidade de medir uma solução. Isso confirma que nossa análise anterior de estimativa de erro estava correta ao prever qual backend executaria o algoritmo com erros mínimos. Para entender melhor o impacto do ruído, também podemos comparar essas probabilidades com a do `GenericBackend`, que simula um cenário ideal e sem ruído.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

Ótimo! Como você pode ver, nosso circuito treinou muito bem, convergindo para um valor que deve corresponder à energia mínima do Hamiltoniano de custo que representa nosso problema. Mas já terminamos? Como podemos ter certeza de que esta é realmente a energia mínima? E igualmente importante, embora possamos ter encontrado a energia mínima, a que estado fundamental ela corresponde? Ou, em outras palavras, qual é a solução real para o problema Max-cut?

Viva! Encontramos com sucesso a solução correta do problema Max-cut usando QAOA!

In [ ]:
def fold_global_circuit(circuit, scale_factor):
    """
    Recebe um QuantumCircuit e um fator de escala (ímpar >= 1) e retorna um novo circuito
    com folding global (ZNE): U (U†U)^k, onde k = (scale_factor - 1) // 2.
    As medições devem estar apenas no final.
    """
    from qiskit import QuantumCircuit
    import copy

    if scale_factor < 1 or scale_factor % 2 != 1:
        raise ValueError("scale_factor deve ser um inteiro ímpar >= 1")
    k = (scale_factor - 1) // 2

    # Remove medições do circuito original
    clean = circuit.remove_final_measurements(inplace=False)

    # Novo circuito com mesmo número de qubits e bits clássicos
    qc = QuantumCircuit(circuit.num_qubits, circuit.num_clbits)

    # Adiciona U
    qc.append(clean, qc.qubits)

    # Adiciona (U†U)^k
    for _ in range(k):
        qc.append(clean.inverse(), qc.qubits)
        qc.append(clean, qc.qubits)

    # Adiciona medições finais (na mesma ordem do circuito original)
    for inst, qargs, cargs in circuit.data:
        if inst.name == "measure":
            qc.measure(qargs[0], cargs[0])

    return qc

Após a rotina de otimização ser executada, podemos ver como a função de custo evolui com o número de iterações para verificar como o algoritmo convergiu.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cost_hamiltonian)
ground_energy = min(eigenvalues).real
num_solutions = eigenvalues.tolist().count(ground_energy)
index_solutions = np.where(eigenvalues == ground_energy)[0].tolist()
print(f"The ground energy of the Hamiltonian is {ground_energy}")
print(f"The number of solutions of the problem is {num_solutions}")
print(f"The list of the solutions based on their index is {index_solutions}")

Parece que temos 8 estados diferentes com probabilidades significativamente maiores que o resto, o que sugere que pode haver 8 estados fundamentais diferentes. Mas como podemos verificar isso? Felizmente para nós, este problema de Max-cut não é muito grande, então ainda pode ser resolvido analiticamente para nos ajudar a discernir se nossa solução é uma boa ou má aproximação. Vamos dar uma olhada!

In [ ]:
def decimal_to_binary(decimal_list, n):
    return [bin(num)[2:].zfill(n) for num in decimal_list]


# Convert the solutions to quantum states
states_solutions = decimal_to_binary(index_solutions, n)
# Sort the dictionary items by their counts in descending order
sorted_states = sorted(counts_list.items(), key=lambda item: item[1], reverse=True)
# Take the top 'num_solutions' entries
top_states = sorted_states[:num_solutions]
# Extract only the states keys from the top entries
qaoa_ground_states = sorted([state for state, count in top_states])
print(f"The analytical solutions for the Max-cut problem are: {states_solutions}")
print(f"The QAOA ground states solutions for the Max-cut are: {qaoa_ground_states}")

Para resolver este problema de Max-cut analiticamente, diagonalizamos o Hamiltoniano de custo. Note que isso pode ser feito porque o tamanho do problema não é muito grande; no entanto, a complexidade deste problema escala exponencialmente com o número de nós (qubits), então o problema se torna intratável para um grande número de nós.

# 3. Simulador quântico com ruído <a id="noisy-simulator"></a>

Com tudo funcionando bem, agora podemos avaliar como o ruído afeta nossos resultados, comparando as execuções em um simulador quântico sem ruído e um simulador quântico com ruído!

## 3.1 Escolhendo o backend <a id="choosing-backend"></a>

O primeiro passo é escolher um backend apropriado. Para esta demonstração, carregaremos uma instância do simulador [AerSimulator](https://quantum.cloud.ibm.com/docs/guides/AerSimulator) que está configurado para simular o ruído de dispositivos quânticos reais. Especificamente, usaremos o modelo de ruído de um dos dispositivos quânticos disponíveis na IBM Quantum Network.

In [ ]:
real_backends = service.backends()
print(f"Os computadores quânticos disponíveis para você são {real_backends}")

# 5. Mitigação de Erros (EM) <a id="em"></a>

Uma das principais áreas de pesquisa para lidar com o ruído inevitável em dispositivos quânticos é a **mitigação de erros (EM)**. A EM consiste em um conjunto de técnicas inteligentes projetadas para reduzir o impacto do ruído sem exigir códigos de correção de erros complexos ou qubits adicionais, recursos que permanecem limitados no hardware quântico de hoje. Em vez de corrigir erros à medida que ocorrem, a EM usa estratégias como repetição de loops, ajustes baseados em calibração e pós-processamento clássico para melhorar a qualidade dos resultados finais, levando a um desempenho aprimorado em nossos algoritmos quânticos.

## 5.1 Extrapolação de Ruído Zero (ZNE) <a id="zne"></a>

ZNE é uma poderosa técnica de mitigação de erros quânticos projetada para reduzir o impacto do ruído em circuitos quânticos sem exigir qubits adicionais ou correção de erros completa. O processo consiste em três estágios essenciais: amplificação de ruído, execução em vários níveis de ruído e extrapolação clássica de volta ao limite de ruído zero.

In [ ]:
# backends=[service.backend("alt_brisbane"),service.backend("alt_kawasaki"),service.backend("alt_torino")]
real_backends = [
    service.backend("ibm_brisbane"),
    service.backend("ibm_sherbrooke"),
    service.backend("ibm_torino"),
]

In [ ]:
noisy_fake_backends = []
for backend in real_backends:
    noisy_fake_backends.append(AerSimulator.from_backend(backend, seed_simulator=seed))
print(f"Os simuladores com ruído são {noisy_fake_backends}")

# Conclusões <a id="conclusions"></a>

Neste laboratório, exploramos as várias fontes de ruído que podem afetar seu circuito quântico e - mais importante - as ferramentas dedicadas que podemos usar para *superar o ruído*. Com essas ferramentas em mãos, você agora está bem equipado para enfrentar qualquer um de seus algoritmos quânticos em andamento e executá-los em hardware quântico real. Lembre-se, os principais passos a seguir são:

- **Entenda seu dispositivo**: conheça as propriedades do hardware, como tempos de coerência, fidelidades de porta e erros de leitura para escolher os melhores qubits.
- **Transpilação inteligente**: mapeie seu circuito lógico para o hardware físico de uma forma que minimize os erros, especialmente para portas de dois qubits.
- **Aplique mitigação e supressão de erros** para reduzir ainda mais o impacto do ruído e melhorar o desempenho.

A probabilidade de medir uma solução correta no `GenericBackend` sem ruído é, como esperado, maior do que nos backends com ruído. No entanto, a diferença não é tão grande. Isso destaca um ponto importante. Mesmo com os atuais computadores quânticos com ruído, desde que sejamos cuidadosos e inteligentes no projeto de nosso circuito e usemos as ferramentas certas, ainda podemos resolver muitos problemas com um bom grau de precisão!

# Desafio Bônus: Ampliando! <a id="bonus"></a>

Como bônus, incluímos uma versão mais complicada deste problema, na qual você implementará o problema Max-cut em hardware quântico da IBM. Esta é a sua chance de colocar em prática tudo o que aprendeu, incluindo as técnicas de mitigação de erros!

In [ ]:
def accumulated_errors(backend: QiskitRuntimeService.backend, circuit: QuantumCircuit) -> list:
    """Compute accumulated gate and readout errors for a given circuit on a specific backend."""
    acc_single_qubit_error = 0
    acc_two_qubit_error = 0
    single_qubit_gate_count = 0
    two_qubit_gate_count = 0
    acc_readout_error = 0
    properties = backend.properties()
    
    # Identify qubits that have measurement operations
    measured_qubits = set()
    for instruction in circuit.data:
        if instruction.operation.name == "measure":
            measured_qubits.add(instruction.qubits[0]._index)

    # Readout error only for measured qubits
    for q in measured_qubits:
        acc_readout_error += properties.readout_error(q)

    # Identify two-qubit gate for the backend
    basis_gates = backend.configuration().basis_gates
    if "ecr" in basis_gates:
        two_qubit_gate = "ecr"
    elif "cz" in basis_gates:
        two_qubit_gate = "cz"
    elif "cx" in basis_gates:
        two_qubit_gate = "cx"
    else:
        two_qubit_gate = None

    # Loop over instructions in circuit.data
    for instruction in circuit.data:
        op = instruction.operation
        qubits = [q._index for q in instruction.qubits]
        # Single-qubit gates (not measure)
        if op.num_qubits == 1 and op.name != "measure":
            single_qubit_gate_count += 1
            acc_single_qubit_error += properties.gate_error(gate=op.name, qubits=qubits[0])
        # Two-qubit gates
        elif op.num_qubits == 2 and op.name == two_qubit_gate:
            two_qubit_gate_count += 1
            acc_two_qubit_error += properties.gate_error(gate=two_qubit_gate, qubits=qubits)

    acc_total_error = acc_two_qubit_error + acc_single_qubit_error + acc_readout_error
    results = [
        acc_total_error,
        acc_two_qubit_error,
        acc_single_qubit_error,
        acc_readout_error,
        single_qubit_gate_count,
        two_qubit_gate_count,
    ]
    return results

Não treinaremos o circuito QAOA em hardware, pois isso consumirá muito do seu plano Open Plan. Em vez disso, executaremos o estimador com os parâmetros otimizados e aplicaremos técnicas de mitigação de erros posteriormente para ver como podemos melhorar os resultados.

Primeiro, calculamos a energia do estado fundamental em hardware sem mitigação de erros.

In [ ]:
qaoa_transpiled_list = []
errors_and_counts_list = []
for noisy_fake_backend in noisy_fake_backends:
    pm = generate_preset_pass_manager(
        backend=noisy_fake_backend,
        optimization_level=3,
        seed_transpiler=seed,
    )
    circuit = pm.run(qaoa_circuit)
    qaoa_transpiled_list.append(circuit)

    errors_and_counts = accumulated_errors(noisy_fake_backend, circuit)
    errors_and_counts_list.append(errors_and_counts)
# You can print your results to visualize if they are correct
for backend, (
    acc_total_error,
    acc_two_qubit_error,
    acc_single_qubit_error,
    acc_readout_error,
    single_qubit_gate_count,
    two_qubit_gate_count,
) in zip(noisy_fake_backends, errors_and_counts_list):
    print(f"Backend {backend.name}")
    print(f"Accumulated two-qubit error of {two_qubit_gate_count} gates: {acc_two_qubit_error:.3f}")
    print(
        f"Accumulated one-qubit error of {single_qubit_gate_count} gates: {acc_single_qubit_error:.3f}"
    )
    print(f"Accumulated readout error: {acc_readout_error:.3f}")
    print(f"Accumulated total error: {acc_total_error:.3f}\n")

Vamos plotar os resultados.

In [ ]:
plot_backend_errors_and_counts(noisy_fake_backends, errors_and_counts_list)

In [ ]:
# Submit your answer using the following code
grade_lab2_ex3(accumulated_errors)

Agora aplicamos diferentes técnicas de mitigação e supressão de erros.

- **Pauli twirling**: esta técnica de mitigação de erros transforma ruído arbitrário em ruído de Pauli mais simples, aplicando e desfazendo aleatoriamente portas de Pauli em torno das operações, reduzindo o impacto de erros coerentes e permitindo uma modelagem e correção de ruído mais eficazes.
- **TREX**: Twirled Readout Error eXtinction é uma técnica de mitigação de erros de leitura que reduz os erros de medição, invertendo aleatoriamente os qubits antes da medição e corrigindo classicamente os resultados, diagonalizando efetivamente a matriz de erros de leitura e simplificando sua inversão para uma estimativa mais precisa do valor esperado.
- **ZNE**: como explicado anteriormente, ZNE é uma técnica de mitigação de erros que estima o resultado de uma computação quântica como se fosse executada em um dispositivo sem ruído. Para isso, aumenta deliberadamente o ruído de forma controlada, mede os resultados e, em seguida, extrapola de volta para o limite de ruído zero.

Agora podemos usar o primitivo Sampler em hardware real sem nenhuma técnica de mitigação e supressão de erros.

In [ ]:
opt_params_list = []
counts_list_backends = []
for noisy_fake_backend, circuit in zip(noisy_fake_backends[:1], qaoa_transpiled_list[:1]):
    result_backend, _ = train_qaoa(init_params, circuit, cost_hamiltonian, noisy_fake_backend)
    opt_params = result_backend.x
    opt_params_list.append(opt_params)
    counts_list_backend = sample_qaoa(opt_params, circuit, noisy_fake_backend)
    counts_list_backends.append(counts_list_backend)

<div class="alert alert-block alert-warning">
<b>Warning: 8 minutes needed. </b>

Running the following code will take approximately 8 minutes to execute, and will block this notebook during that time. 

Se você quiser pular esta célula, vá diretamente para a [Seção 4](#transpiler).

</div>

In [ ]:
for noisy_fake_backend, circuit in zip(noisy_fake_backends[1:], qaoa_transpiled_list[1:]):
    result_backend, _ = train_qaoa(init_params, circuit, cost_hamiltonian, noisy_fake_backend)
    opt_params = result_backend.x
    opt_params_list.append(opt_params)
    counts_list_backend = sample_qaoa(opt_params, circuit, noisy_fake_backend)
    counts_list_backends.append(counts_list_backend)

# 3.2 Estimando erros usando NEAT <a id="neat"></a>

Até agora, analisamos individualmente as propriedades do dispositivo. Agora, vamos descobrir como estimar o impacto esperado do ruído para um circuito específico. Para isso, usaremos uma ferramenta bastante útil: **Noise Equivalent Application Testing (NEAT)**. 

Do ponto de vista prático, uma maneira de usar o NEAT é simular um circuito quântico em um cenário com e sem ruído, medindo um observável cujo valor esperado é exatamente 1 no caso ideal (sem ruído). Nesta configuração, qualquer desvio de 1 na simulação com ruído reflete diretamente o impacto do ruído no circuito. Uma maneira fácil de garantir que o observável atenda aos requisitos é escolher:

In [ ]:
for noisy_fake_backend, counts_list_backend in zip(noisy_fake_backends, counts_list_backends):
    solutions_counts = [counts_list_backend[key] for key in states_solutions]
    print(
        f"Probability of measuring a solution for {noisy_fake_backend.name} is {float(sum(solutions_counts)/SHOTS)}"
    )

Vamos aplicar o NEAT aos nossos diferentes backends com ruído para ver como eles se saem.

In [ ]:
solutions_counts_noiseless = [counts_list[key] for key in states_solutions]
print(
    f"Probability of measuring a solution for {generic_backend.name} is {float(sum(solutions_counts_noiseless)/SHOTS)}"
)

Primeiro, realizaremos uma transpilação do circuito quântico para um layout no computador quântico que minimize o número de portas de dois qubits, pois essa será a principal fonte de erro.

# 4. Transpilação <a id="transpilation"></a>

Usando a ferramenta NEAT, podemos observar como o valor esperado em uma simulação com ruído se desvia do valor ideal de 1, de uma forma que concorda com a análise de ruído realizada na [Seção 3.1](#choosing-backend). 

No entanto, a transpilação padrão nem sempre é a melhor. Vamos ver se conseguimos fazer melhor!

In [ ]:
results = []
for backend, opt_params in zip(noisy_fake_backends, opt_params_list):
    print(f"\nRunning on backend: {backend.name}")

    # Prepare the QAOA circuit with optimized parameters
    qaoa_neat = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=layers)
    qc = qaoa_neat.assign_parameters(opt_params)

    # Transpile the circuit
    qc_transpiled = generate_preset_pass_manager(
        optimization_level=3,
        basis_gates=backend.configuration().basis_gates[:n],
        seed_transpiler=seed,
    ).run(qc)
    # Use cost Hamiltonian as observable for Cliffordization
    obs = cost_hamiltonian
    analyzer = Neat(backend)
    clifford_pub = analyzer.to_clifford([(qc_transpiled, obs)])[0]

    # Construct observable from Clifford circuit
    state_clifford = Statevector.from_instruction(clifford_pub.circuit)
    obs_clifford = SparsePauliOp.from_operator(Operator(DensityMatrix(state_clifford).data))

    # Apply layout
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1, seed_transpiler=seed)
    isa_qc = pm.run(clifford_pub.circuit)
    isa_obs = obs_clifford.apply_layout(isa_qc.layout)

    # Prepare pubs and simulate
    pubs = [(isa_qc, isa_obs)]
    result_noiseless = (
        analyzer.ideal_sim(pubs, cliffordize=True, seed_simulator=seed)._pub_results[0].vals
    )
    noisy_results = (
        analyzer.noisy_sim(pubs, cliffordize=True, seed_simulator=seed)._pub_results[0].vals
    )

    # Store and print results
    results.append({"backend": backend.name, "noiseless": result_noiseless, "noisy": noisy_results})
    print(f"Ideal results on {backend.name}:\n{result_noiseless:.3f}")
    print(f"Noisy results on {backend.name}:\n{noisy_results:.3f}")

# 4. Transpilador <a id="transpiler"></a>

O transpilador é uma das ferramentas mais importantes e úteis para executar circuitos quânticos em hardware quântico real. Ele serve como uma ponte entre a versão abstrata e idealizada de um circuito quântico e a implementação física no dispositivo quântico real. Quando você projeta um circuito, normalmente usa qubits virtuais e portas ideais sem considerar as limitações específicas do hardware. O transpilador traduz este circuito de alto nível em uma versão que pode ser implementada em um computador quântico real, usando apenas as operações de porta disponíveis nos qubits físicos do dispositivo.

Por exemplo, suponha que seu circuito inclua uma porta CNOT entre os qubits virtuais 0 e 1. Em um dispositivo real, esses dois qubits podem não estar diretamente conectados. Nesses casos, o transpilador insere uma série de portas SWAP para mover os estados quânticos para qubits fisicamente adjacentes, permitindo a operação CNOT. Alternativamente, o transpilador pode encontrar um mapeamento de qubit mais eficiente, de modo que os qubits virtuais 0 e 1 sejam reatribuídos a qubits físicos que estão diretamente conectados - por exemplo, qubits 3 e 4, evitando assim a necessidade de portas SWAP adicionais.

Até agora temos usado o transpilador implicitamente ao executar o gerenciador de passes em `generate_preset_pass_manager`. No entanto, agora vamos nos concentrar em entendê-lo melhor e aproveitá-lo ao máximo para um melhor design de circuito.

Uma das tarefas mais importantes de um transpilador quântico é determinar um layout ótimo de qubits para executar um circuito quântico. Isso envolve encontrar o melhor mapeamento entre os qubits virtuais do circuito e os qubits físicos do dispositivo. Para fazer isso, há algumas coisas a serem consideradas.

Primeiro, o transpilador deve verificar todas as portas de dois qubits no circuito e garantir que os qubits físicos selecionados estejam conectados de uma forma que permita essas operações e reduza a necessidade de usar portas SWAP adicionais. Isso requer inspecionar o mapa de acoplamento, que mostra como os qubits físicos estão conectados.

Primeiro realizaremos uma transpilação do circuito quântico para um layout no computador quântico que minimize o número de portas de dois qubits, já que essa será a principal fonte de erro.

Em particular, consideraremos o backend `ibm_brisbane`, onde, como mostrado na [Seção 3.1](#choosing-backend), os erros de porta de dois qubits respondem pela maioria do erro total acumulado.

In [ ]:
# We select the `ibm_brisbane` backend
num_backend = 0
noisy_fake_backend = noisy_fake_backends[num_backend]

pm = generate_preset_pass_manager(
    backend=noisy_fake_backend,
    optimization_level=3,
    seed_transpiler=seed,
    layout_method="sabre",
)
circuit_transpiled = pm.run(qaoa_circuit)


def two_qubit_gate_errors_per_circuit_layout(
    circuit: QuantumCircuit, backend: QiskitRuntimeService.backend
) -> tuple:
    """Calculate accumulated two-qubit gate errors and related metrics for a given circuit layout."""
    pair_list = []
    error_pair_list = []
    error_acc_pair_list = []
    two_qubit_gate_count = 0
    properties = backend.properties()
    if "ecr" in (backend.configuration().basis_gates):
        two_qubit_gate = "ecr"
    elif "cz" in (backend.configuration().basis_gates):
        two_qubit_gate = "cz"
    for instruction in circuit.data:
        if instruction.operation.num_qubits == 2:
            two_qubit_gate_count += 1
            pair = [instruction.qubits[0]._index, instruction.qubits[1]._index]
            error_pair = properties.gate_error(gate=two_qubit_gate, qubits=pair)
            if pair not in (pair_list):
                pair_list.append(pair)
                error_pair_list.append(error_pair)
                error_acc_pair_list.append(error_pair)
            else:
                pos = pair_list.index(pair)
                error_acc_pair_list[pos] += error_pair

    acc_two_qubit_error = sum(error_acc_pair_list)
    return (
        acc_two_qubit_error,
        two_qubit_gate_count,
        pair_list,
        error_pair_list,
        error_acc_pair_list,
    )


(
    acc_two_qubit_error,
    two_qubit_gate_count,
    pair_list,
    error_pair_list,
    error_acc_pair_list,
) = two_qubit_gate_errors_per_circuit_layout(circuit_transpiled, noisy_fake_backend)
two_qubit_ops_list = [int(a / b) for a, b in zip(error_acc_pair_list, error_pair_list)]
# We print the results
print(f"The pairs of qubits that need to perform two-qubit operations are:\n {pair_list}")
print(
    f"The errors introduced by each of the two-qubit operations are:\n {[round(err,3) for err in error_pair_list]}"
)
print(
    f"The accumulated errors introduced by each of the two-qubit operations are:\n {[round(err,3) for err in error_acc_pair_list]}"
)
print(f"The repetitions of each one of the two-qubit operations is:\n {two_qubit_ops_list}")
print(f"The number of two-qubit operations in total:\n {two_qubit_gate_count}")
print(f"The total accumulated error by two-qubit operations is:\n {acc_two_qubit_error:.3f}")

Existem dois tipos comuns de folding: **Folding Global** e **Folding Local**.

Nesta seção vimos que a transpilação é um passo fundamental na preparação de circuitos quânticos para serem executados em hardware real. Como diferentes dispositivos têm diferentes layouts e conjuntos de portas, um bom transpilador ajuda a adaptar seu circuito para se adequar ao hardware enquanto tenta manter coisas como profundidade quântica e taxas de erro baixas, alcançando assim melhor desempenho de seus algoritmos quânticos.

In [ ]:
# We build a graph with the connectivity constraints of our backend that includes the two-qubit gate errors as weights in the edges
graph = rx.PyDiGraph()
graph.add_nodes_from(np.arange(0, noisy_fake_backend.num_qubits, 1))
two_qubit_gate = "ecr"
graph.add_edges_from(
    [
        (
            edge[0],
            edge[1],
            noisy_fake_backend.properties().gate_error(
                gate=two_qubit_gate, qubits=(edge[0], edge[1])
            ),
        )
        for edge in noisy_fake_backend.coupling_map
    ]
)
draw_graph(graph, node_size=150, with_labels=True, width=1)

Neste quinto exercício você usará o `layout_method` `sabre` para identificar o layout que tem o menor erro acumulado de portas de dois qubits.

Para fazer isso, realize uma varredura sobre os valores de `seed_transpiler` de 0 a 500 e use o `generate_preset_pass_manager` com `optimization_level=3` para selecionar o melhor layout. Lembre-se de que você pode contar o erro acumulado das portas de dois qubits e o número de portas de dois qubits usando a função `two_qubit_gate_errors_per_circuit_layout`.

In [ ]:
def remap_nodes(original_labels: list, edge_list: list[list]) -> list[list[int]]:
    """Remap node labels to a new sequence starting from 0 based on their order in original_labels."""
    label_mapping = {label: idx for idx, label in enumerate(original_labels)}
    remapped = [[label_mapping[src], label_mapping[dst]] for src, dst in edge_list]
    return remapped


layout_list = list(circuit_transpiled.layout.initial_layout.get_physical_bits().keys())[:5]
logical_pair_list = remap_nodes(layout_list, pair_list)
print(f"Physical qubit layout list:\n {layout_list}")
print(f"\nOriginal two-qubit gates list:\n {pair_list}")
print(f"\nRemapped two-qubit gates list (in logical qubits):\n {logical_pair_list}")

#### Folding Local

O folding local foca em aumentar seletivamente o ruído em partes específicas do circuito quântico, geralmente em torno das portas ou subcircuitos mais propensos a erros. Em vez de dobrar todo o circuito como um bloco, este método foca em portas quânticas individuais ou grupos de portas. Isso fornece maior controle e calibração mais precisa do processo de amplificação de ruído. Sua operação é pegar de cada circuito quântico cada porta quântica $G$. Podemos aplicar folding local envolvendo-a com sua operação inversa da seguinte forma:

$$G \rightarrow G G^\dagger G$$

<b>Exercício 6b: Folding Local</b>

**Seu Objetivo:** Implementar folding local em circuitos quânticos.

Nesta segunda parte do Exercício 6 (Seção b), sua tarefa é escrever uma função que aplica folding local a um circuito quântico. Ao contrário do folding global, este método foca em portas individuais e as dobra seletivamente para amplificar o ruído em áreas específicas do circuito. A função deve permitir controle flexível das portas sendo dobradas (por exemplo, não podemos dobrar uma porta de medição) e sua avaliação em diferentes escalas de amplificação de ruído.

In [ ]:
def find_paths_with_weight_sum_below_threshold(
    graph: rx.PyDiGraph,
    threshold: float,
    two_qubit_ops_list: list[int],
    logical_pair_list: list[list[int]],
) -> tuple[list[list[int]], list[float]]:
    """Find all valid paths through a graph whose weighted sum is below a given threshold."""
    valid_paths = []
    valid_weights = []
    for start_node in range(graph.num_nodes()):
        paths = [[start_node]]
        weights = [0]
        for i in range(len(two_qubit_ops_list)):
            new_paths = []
            new_weights = []
            for path, weight in zip(paths, weights):
                if logical_pair_list[i][0] < logical_pair_list[i][1]:
                    index_of_expanding_node = logical_pair_list[i][0]
                    node_to_expand_from = path[index_of_expanding_node]
                    for neighbor in graph.neighbors(node_to_expand_from):
                        if neighbor not in path and graph.has_edge(node_to_expand_from, neighbor):
                            edge_weight = graph.get_edge_data(node_to_expand_from, neighbor) * two_qubit_ops_list[i]
                            new_paths.append(path + [neighbor])
                            new_weights.append(weight + edge_weight)
                else:
                    index_of_expanding_node = logical_pair_list[i][1]
                    node_to_expand_from = path[index_of_expanding_node]
                    for neighbor in graph.neighbors_undirected(node_to_expand_from):
                        if neighbor not in path and graph.has_edge(neighbor, node_to_expand_from):
                            edge_weight = graph.get_edge_data(neighbor, node_to_expand_from) * two_qubit_ops_list[i]
                            new_paths.append(path + [neighbor])
                            new_weights.append(weight + edge_weight)
            paths = new_paths
            weights = new_weights
        for path, weight in zip(paths, weights):
            if weight < threshold:
                valid_paths.append(path)
                valid_weights.append(weight)
    return valid_paths, valid_weights

In [ ]:
# Define the optimal layout based on the previous analysis
opt_layout = [72, 62, 81, 61, 63]  # Physical qubit layout from previous cell

init_layout = Layout({q: phys for q, phys in zip(qaoa_circuit.qubits, opt_layout)})
pm = generate_preset_pass_manager(
    backend=noisy_fake_backend,
    optimization_level=3,
    seed_transpiler=seed,
    initial_layout=init_layout,
    layout_method="sabre",
)

circuit_opt = pm.run(qaoa_circuit)
circuit_opt.draw(idle_wires=False, fold=False)

In [ ]:
# Submit your answer using the following code
grade_lab2_ex4(find_paths_with_weight_sum_below_threshold)

Depois de amplificar o ruído através do folding, o circuito quântico é executado várias vezes em diferentes níveis de ruído. Finalmente, no terceiro passo, **Extrapolação**, os resultados das execuções ruidosas são pós-processados usando métodos de ajuste clássicos, como extrapolação linear, polinomial ou exponencial, para estimar qual seria o resultado em um cenário hipotético de ruído zero. Através deste pipeline, ZNE ajuda a recuperar resultados de maior fidelidade do hardware quântico ruidoso atual, tornando-se uma ferramenta valiosa no cenário de computação quântica de curto prazo.

Para validar e analisar os resultados da ZNE que acabamos de implementar, podemos aplicar três tipos de métodos de extrapolação: linear, polinomial e exponencial. Essas abordagens ajudam a estimar a saída do circuito no limite de ruído zero com base nos dados coletados em níveis de ruído mais altos. Para ilustrar como isso pode ser feito, considere o seguinte código de exemplo:

<a id="exercise_5"></a>
<div class="alert alert-block alert-success">
    
<b>Exercício 5: Melhor Mapeamento </b> 

**Seu Objetivo:** Use o transpilador para encontrar o layout ótimo.

Neste quinto exercício você usará o `layout_method` `sabre` para identificar o layout que tem o menor erro acumulado de portas de dois qubits. <br>

Para fazer isso, realize uma varredura sobre os valores de `seed_transpiler` de 0 a 500 e use o `generate_preset_pass_manager` com `optimization_level=3` para selecionar o melhor layout. Lembre-se de que você pode contar o erro acumulado das portas de dois qubits e o número de portas de dois qubits usando a função `two_qubit_gate_errors_per_circuit_layout`.
</div>

In [ ]:
def finding_best_seed(
    circuit: QuantumCircuit, backend: QiskitRuntimeService.backend
) -> tuple[QuantumCircuit, int, float, int]:
    """Find the transpiler seed that minimizes two-qubit gate error for a given circuit and backend."""

    # We initialize the minimum error accumulated
    min_err_acc_seed_loop = 100
    circuit_opt_best_seed = None
    best_seed_transpiler = None
    two_qubit_gate_count_seed_loop = None
    # First we loop over 500 seeds and transpile the circuit
    for seed_transpiler in range(0, 500):
        pm = generate_preset_pass_manager(
            backend=backend,
            optimization_level=3,
            seed_transpiler=seed_transpiler,
            layout_method="sabre",
        )
        circuit_opt_seed = pm.run([circuit])[0]
        # ---- TODO : Task 5 ---
        # Goal: Find the transpiler seed that minimizes two-qubit gate error for a given circuit and backend looping from 0 to 500

        # Use the `two_qubit_gate_errors_per_circuit_layout` function to count for the error of the transpile circuit
        acc_two_qubit_error, two_qubit_gate_count, _, _, _ = two_qubit_gate_errors_per_circuit_layout(circuit_opt_seed, backend)

        # Check if the error accounted above is smaller than min_err_acc_seed_loop. If so, assign the variables that this function returns
        if acc_two_qubit_error < min_err_acc_seed_loop:
            min_err_acc_seed_loop = acc_two_qubit_error
            circuit_opt_best_seed = circuit_opt_seed
            best_seed_transpiler = seed_transpiler
            two_qubit_gate_count_seed_loop = two_qubit_gate_count

        # --- End of TODO ---

    return (
        circuit_opt_best_seed,
        best_seed_transpiler,
        min_err_acc_seed_loop,
        two_qubit_gate_count_seed_loop,
    )

In [ ]:
(
    circuit_opt_seed_loop,
    best_seed_transpiler,
    min_err_acc_seed_loop,
    two_qubit_gate_count_seed_loop,
) = finding_best_seed(qaoa_circuit, noisy_fake_backend)

best_layout = list(circuit_opt_seed_loop.layout.initial_layout.get_physical_bits().keys())[:n]
print(f"Best transpiler seed: {best_seed_transpiler}")
print(f"Minimum accumulated two-qubit gate error: {min_err_acc_seed_loop:.3f}")
print(f"Two-qubit gate count for best seed: {two_qubit_gate_count_seed_loop}")
print(f"Best layout (first n logical qubits mapped to physical qubits):\n {best_layout}")

In [ ]:
# Submit your answer using the following code
grade_lab2_ex5(finding_best_seed)

Em seguida, amostramos o circuito QAOA com esses circuitos para destacar a importância de uma boa transpilação.

<div class="alert alert-block alert-warning">
<b>Warning: 5 minutes needed.</b>

Running the following code will take approximately 5 minutes to execute, and will block this notebook during that time.  You can skip to the next cell.

</div>

In [ ]:
counts_list_transpiled_circuits = []
circuit_transpiled_list = [circuit_transpiled, circuit_opt_seed_loop]
opt_params_list_transpiled_circuits = []
for circuit in circuit_transpiled_list:
    result_backend, _ = train_qaoa(init_params, circuit, cost_hamiltonian, noisy_fake_backend)
    opt_params = result_backend.x
    opt_params_list_transpiled_circuits.append(opt_params)
    counts_list_transpiled_circuit = sample_qaoa(opt_params, circuit, noisy_fake_backend)
    counts_list_transpiled_circuits.append(counts_list_transpiled_circuits)
    solutions_counts = [counts_list_transpiled_circuit[key] for key in states_solutions]
    print(f"Probability of measuring a solution for is {float(sum(solutions_counts)/SHOTS)}")

No seguinte, lembre-se de usar o método de layout `sabre` e fazer um loop sobre diferentes seeds para minimizar uma métrica específica - como contagem de portas de dois qubits, profundidade ou erro acumulado - para maximizar o desempenho.

Nesta seção vimos que a transpilação é um passo fundamental na preparação de circuitos quânticos para serem executados em hardware real. Como diferentes dispositivos têm diferentes layouts e conjuntos de portas, um bom transpilador ajuda a adaptar seu circuito para se adequar ao hardware enquanto tenta manter baixos fatores como profundidade quântica e taxas de erro, alcançando assim melhor desempenho de seus algoritmos quânticos.

# 5. Mitigação de Erros (EM) <a id="em"></a>

Uma das principais áreas de pesquisa para abordar o inevitável ruído em dispositivos quânticos é a **mitigação de erros (EM)**. EM consiste em um conjunto de técnicas inteligentes projetadas para reduzir o impacto do ruído sem exigir códigos complexos de correção de erro ou qubits adicionais, recursos que permanecem limitados no hardware quântico atual. Em vez de corrigir erros conforme eles ocorrem, EM usa estratégias como repetição de loops, ajustes baseados em calibração e pós-processamento clássico para melhorar a qualidade dos resultados finais, levando a um desempenho aprimorado em nossos algoritmos quânticos.

Um dos métodos mais amplamente utilizados é a Extrapolação de Ruído Zero (ZNE). Nesta abordagem, o mesmo circuito quântico é executado múltiplas vezes com níveis de ruído deliberadamente aumentados. Então, técnicas de extrapolação matemática são aplicadas para estimar qual teria sido o resultado na ausência de ruído. Este método foi introduzido por [Temme et al. em 2017](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.119.180509).

In [73]:
def fold_global_circuit(circuit: QuantumCircuit, scale_factor: int) -> QuantumCircuit:
    """Apply global circuit folding for Zero Noise Extrapolation (ZNE)."""
    if scale_factor % 2 == 0 or scale_factor < 1:
        raise ValueError("scale_factor must be an odd positive integer (1, 3, 5, ...)")
    
    # For scale factor 1, return the original circuit
    if scale_factor == 1:
        return circuit.copy()
    
    # Remove measurements from the original circuit to get clean circuit
    clean_circuit = circuit.remove_final_measurements(inplace=False)
    
    # Create folded circuit - following your backend-optimized approach
    folded_circuit = QuantumCircuit(clean_circuit.num_qubits, circuit.num_clbits)
    
    # Calculate number of repetitions: n_repeat = (scale_factor - 1) // 2
    n_repeat = (scale_factor - 1) // 2
    
    # Apply global folding following your specific pattern:
    # For scale_factor=3: C → C† → C 
    # For scale_factor=5: C → C† → C → C† → C
    
    # Method 1: Direct append with qargs (matching your style)
    # Get available qubits for the circuit
    available_qubits = list(range(min(clean_circuit.num_qubits, folded_circuit.num_qubits)))
    
    # Add original circuit (C)
    folded_circuit.append(clean_circuit, qargs=available_qubits)
    
    # Add the folding pairs (C† C)^n_repeat
    for _ in range(n_repeat):
        # Add C† (inverse circuit)
        folded_circuit.append(clean_circuit.inverse(), qargs=available_qubits) 
        # Add C (original circuit)
        folded_circuit.append(clean_circuit, qargs=available_qubits)
    
    # Add measurements back at the end
    folded_circuit.measure_all()
    
    return folded_circuit

In [74]:
# Submit your answer using the following code
grade_lab2_ex6a(fold_global_circuit)

Submitting your answer. Please wait...
Oops 😕! Hmm not quite: Your folding method is not correct :(
Please review your answer and try again.


#### Folding Local

O folding local foca em aumentar seletivamente o ruído em partes específicas do circuito quântico, geralmente em torno das portas ou subcircuitos mais propensos a erros. Em vez de dobrar todo o circuito como um bloco, este método foca em portas quânticas individuais ou grupos de portas. Isso fornece maior controle e calibração mais precisa do processo de amplificação de ruído. Sua operação é pegar de cada circuito quântico cada porta quântica $G$. Podemos aplicar folding local envolvendo-a com sua operação inversa da seguinte forma:

$$ G \rightarrow G \cdot G^\dagger \cdot G $$

Esta transformação logicamente cancela o par inserido $G^\dagger \cdot G$, deixando a computação inalterada. No entanto, a execução agora envolve três vezes mais operações de porta, aumentando assim a exposição ao ruído local. Isso o torna ideal para simular diferentes níveis de ruído de forma seletiva e sistemática.

<a id="exercise_6b"></a>

<div class="alert alert-block alert-success">
<b>Exercício 6b: Folding Local</b>

**Seu Objetivo:** Implementar folding local em circuitos quânticos.

Nesta segunda parte do Exercício 6 (Seção b), sua tarefa é escrever uma função que aplica folding local a um circuito quântico. Diferentemente do folding global, este método foca em portas individuais e as dobra seletivamente para amplificar o ruído em áreas específicas do circuito. A função deve permitir controle flexível das portas sendo dobradas (por exemplo, não podemos dobrar uma porta de medição) e sua avaliação em diferentes escalas de amplificação de ruído.
</div>

In [ ]:
O primeiro estágio, **Amplificação de Ruído**, está no coração da ZNE. A ideia é executar o circuito quântico em versões que têm mais ruído do que o usual, mas de uma forma controlada e reversível. Comparando como a saída muda conforme o ruído aumenta, torna-se possível inferir qual seria o resultado sem ruído algum. Isso é tipicamente alcançado usando uma técnica chamada folding de circuito.

O folding de circuito aumenta artificialmente o ruído em uma computação quântica inserindo portas adicionais que, em teoria, não alteram o resultado lógico. Essas portas adicionais são as operações adjuntas (inversas) de portas aplicadas anteriormente. Por exemplo, uma operação unitária $U$ pode ser transformada em $ U \cdot U^\dagger \cdot U$, que logicamente ainda computa $U$, mas leva mais tempo para executar, ficando assim exposta a mais ruído do hardware.

In [ ]:
# Submit your answer using the following code
grade_lab2_ex6b(fold_local_circuit)

### Extrapolação

Após amplificar o ruído através do folding, o circuito quântico é executado múltiplas vezes em diferentes níveis de ruído. Finalmente, no terceiro passo, **Extrapolação**, os resultados das execuções ruidosas são pós-processados usando métodos de ajuste clássicos, como extrapolação linear, polinomial ou exponencial, para estimar qual seria o resultado em um cenário hipotético de ruído zero. Através deste pipeline, ZNE ajuda a recuperar resultados de maior fidelidade do hardware quântico ruidoso atual, tornando-se uma ferramenta valiosa no cenário de computação quântica de curto prazo.

Agora, integraremos ZNE na execução de um circuito QAOA, melhorando a precisão dos resultados em hardware quântico ruidoso.

In [ ]:
def basic_zne(
    circuit,
    scales,
    backend,
    opt_params,
    observable,
):
    """ZNE básica (Extrapolação de Ruído Zero) usando folding local."""

    exp_vals = []
    xdata = np.array(scales)
    estimator = Estimator(mode=backend)

    for scale in scales:
        # Aplicar folding local
        folded = fold_local_circuit(circuit, scale)

        # Transpilar para o backend
        basis_gates = backend.target.operation_names
        transpiled_folded = generate_preset_pass_manager(
            basis_gates=basis_gates, optimization_level=0, seed_transpiler=seed
        ).run(folded)
        pub = (
            transpiled_folded,
            observable.apply_layout(circuit.layout),
            opt_params,
        )
        # Avaliar o valor esperado
        job = estimator.run([pub])
        results = job.result()[0]
        exp_vals.append(results.data.evs)

    return xdata, exp_vals, pub

#### Folding Global

No folding global, todo o circuito quântico é dobrado como um único bloco. Isso significa que a operação unitária completa $U$ que o circuito representa é envolvida com sua operação adjunta, produzindo a transformação:

$$U \rightarrow U (U^\dagger U)^k$$

onde $k$ é um número inteiro não negativo que controla o grau de amplificação de ruído.

O folding global é particularmente útil para aplicar rapidamente um aumento uniforme de ruído em todo o circuito. É direto de implementar e não requer conhecimento da estrutura interna do circuito. Como tal, serve como uma abordagem de granularidade grossa para amplificação de ruído, adequada para extrapolação de propósito geral quando o controle fino não é necessário.

<a id="exercise_6a"></a>

<b>Exercício 6a: Folding Global</b>

**Seu Objetivo:** Implementar folding global nos circuitos quânticos.

Nesta parte do Exercício 6 (Seção a), você criará uma função que aplica folding global a qualquer circuito quântico. Sua implementação deve permitir a avaliação do circuito em diferentes fatores de escala de ruído, que simulam níveis aumentados de ruído preservando a saída lógica do circuito. O fator de escala de ruído representa o número de vezes que um circuito $U$ ou $U^\dagger$ é aplicado, sendo 1 o caso sem folding.

In [ ]:
methods = ["linear", "quadratic", "exponential"]

for method in methods:
    print(f"\n Extrapolation Method: {method.upper()}")

    # Perform the extrapolation
    zero_val, fitted_vals, fit_params, fit_fn = zne_method(method=method, xdata=xdata, ydata=ydata)

    # Print the extrapolated expectation value
    print(f"⟨Z⟩ (ZNE Estimate): {zero_val:.3f}")

    # Plot the results
    plot_zne(xdata, fitted_vals, zero_val, fit_fn, fit_params, method)

#### Folding Local

O folding local foca em aumentar seletivamente o ruído em partes específicas do circuito quântico, geralmente em torno das portas ou subcircuitos mais propensos a erros. Em vez de dobrar todo o circuito como um bloco, este método foca em portas quânticas individuais ou grupos de portas. Isso fornece maior controle e calibração mais precisa do processo de amplificação de ruído. Sua operação é pegar de cada porta quântica $G$ do circuito quântico. Podemos aplicar folding local envolvendo-a com sua operação inversa da seguinte forma:

$$G \rightarrow G (G^\dagger G)^k$$

onde $k$ é um número inteiro não negativo que controla quantas vezes a porta é dobrada.

In [ ]:
# Check your submission status with the code below
from qc_grader.grader.grade import check_lab_completion_status

check_lab_completion_status("qgss_2025")

<a id="exercise_6b"></a>

<b>Exercício 6b: Folding Local</b>

**Seu Objetivo:** Implementar folding local em circuitos quânticos.

Nesta segunda parte do Exercício 6 (Seção b), sua tarefa é escrever uma função que aplica folding local a um circuito quântico. Diferentemente do folding global, este método foca em portas individuais e as dobra seletivamente para amplificar o ruído em áreas específicas do circuito. A função deve permitir controle flexível das portas sendo dobradas (por exemplo, não podemos dobrar uma porta de medição) e sua avaliação em diferentes escalas de amplificação de ruído.

In [ ]:
# We define the number of nodes:
n_ext = 7
# We define the graph
graph_ext = rx.PyGraph()
graph_ext.add_nodes_from(np.arange(0, n_ext, 1))
generic_backend_ext = GenericBackendV2(n_ext, seed=seed)
weights = 1
# We make it explicitly asymmetrical to have a smaller set of solutions
graph_ext.add_edges_from(
    [(edge[0], edge[1], weights) for edge in generic_backend_ext.coupling_map][:-7]
)
draw_graph(graph_ext, node_size=200, with_labels=True, width=1)
max_cut_paulis_ext = graph_to_Pauli(graph_ext)
cost_hamiltonian_ext = SparsePauliOp.from_list(max_cut_paulis_ext)
pm = generate_preset_pass_manager(
    optimization_level=3, backend=generic_backend_ext, seed_transpiler=seed
)
layers = 2
init_params = np.zeros(2 * layers)
qaoa_circuit_ext = QAOAAnsatz(cost_operator=cost_hamiltonian_ext, reps=layers)
qaoa_circuit_ext.measure_all()
qaoa_circuit_ext_transpiled = pm.run(qaoa_circuit_ext)

### Escolha do backend

In [ ]:
def accumulated_errors(circuit, properties):
    """
    Calculate the accumulated error rates for a quantum circuit.
    
    Args:
        circuit: The quantum circuit to analyze
        properties: Backend properties containing error information
        
    Returns:
        Tuple of (single_qubit_error, two_qubit_error, readout_error, total_error)
    """
    
    single_qubit_gate_count = 0
    two_qubit_gate_count = 0
    acc_single_qubit_error = 0.0
    acc_two_qubit_error = 0.0
    acc_readout_error = 0.0
    
    # Determine the two-qubit gate type based on basis gates
    basis_gates = properties.gates
    basis_gate_names = [gate.gate for gate in basis_gates]
    
    if "ecr" in basis_gate_names:
        two_qubit_gate = "ecr"
    elif "cz" in basis_gate_names:
        two_qubit_gate = "cz"
    elif "cx" in basis_gate_names:
        two_qubit_gate = "cx"
    else:
        two_qubit_gate = None

    # Loop over instructions in circuit.data
    for instruction in circuit.data:
        op = instruction.operation
        qubits = [q._index for q in instruction.qubits]
        # Single-qubit gates (not measure)
        if op.num_qubits == 1 and op.name != "measure":
            single_qubit_gate_count += 1
            acc_single_qubit_error += properties.gate_error(gate=op.name, qubits=qubits[0])
        # Two-qubit gates
        elif op.num_qubits == 2 and op.name == two_qubit_gate:
            two_qubit_gate_count += 1
            acc_two_qubit_error += properties.gate_error(gate=op.name, qubits=qubits)
        # Measurement operations
        elif op.name == "measure":
            qubit_index = qubits[0]
            acc_readout_error += properties.readout_error(qubit_index)

    acc_total_error = acc_single_qubit_error + acc_two_qubit_error + acc_readout_error

    return acc_single_qubit_error, acc_two_qubit_error, acc_readout_error, acc_total_error

Em seguida, amostramos o circuito QAOA com esses circuitos para destacar a importância de uma boa transpilação.

In [ ]:
circuit_ext_opt_seed, best_seed_transpiler, min_err_acc_seed, _ = finding_best_seed(
    qaoa_circuit_ext, best_backend_ext
)
print(f"Best transpiler seed: {best_seed_transpiler}")
print(f"Minimum accumulated two-qubit gate error: {min_err_acc_seed:.3f}")

### Estimator

Não iremos treinar o circuito QAOA em hardware, pois isso consumirá muito da sua cota do Open Plan. Em vez disso, executaremos o estimador com os parâmetros otimizados e aplicaremos técnicas de mitigação de erros posteriormente para ver como podemos melhorar os resultados.

In [ ]:
best_backend_sim = AerSimulator.from_backend(best_backend_ext, seed_simulator=seed)
result_qaoa_sim, objective_func_vals_sim = train_qaoa(
    init_params, circuit_ext_opt_seed, cost_hamiltonian_ext, best_backend_sim
)

Agora podemos amostrar do circuito QAOA.

In [ ]:
opt_params_sim = result_qaoa_sim.x
counts_list_sim = sample_qaoa(opt_params_sim, circuit_ext_opt_seed, best_backend_sim)

# Verificando os resultados

In [ ]:
eigenvalues_ext, eigenvectors_ext = np.linalg.eig(cost_hamiltonian_ext)
ground_energy_ext = min(eigenvalues_ext).real
num_solutions_ext = eigenvalues_ext.tolist().count(ground_energy_ext)
index_solutions_ext = np.where(eigenvalues_ext == ground_energy_ext)[0].tolist()
print(f"The ground energy of the Hamiltonian is {ground_energy_ext}")
print(f"The number of solutions of the problem is {num_solutions_ext}")
print(f"The list of the solutions based on their index is {index_solutions_ext}")

states_solutions_ext = decimal_to_binary(index_solutions_ext, n_ext)
# Sort the dictionary items by their counts in descending order
sorted_states_sim = sorted(counts_list_sim.items(), key=lambda item: item[1], reverse=True)
# Take the top 'num_solutions' entries
top_states_sim = sorted_states_sim[:num_solutions_ext]
# Extract only the states keys from the top entries
qaoa_ground_states_sim = sorted([state for state, count in top_states_sim])
print(f"The analytical solutions for the Max-cut problem are: {states_solutions_ext}")
print(
    f"The QAOA ground states solutions for the Max-cut are: {qaoa_ground_states_sim} for {best_backend_sim.name}"
)

Nesta seção, vimos que a transpilação é um passo fundamental na preparação de circuitos quânticos para serem executados em hardware real. Como diferentes dispositivos têm diferentes layouts e conjuntos de portas, um bom transpilador ajuda a adaptar seu circuito para se adequar ao hardware, tentando manter baixos fatores como profundidade quântica e taxas de erro, alcançando assim um melhor desempenho de seus algoritmos quânticos.

### Estimador

Não iremos treinar o circuito QAOA em hardware, pois isso consumirá muito da sua cota do Open Plan. Em vez disso, executaremos o estimador com os parâmetros otimizados e aplicaremos técnicas de mitigação de erros posteriormente para ver como podemos melhorar os resultados.

Primeiro calculamos a energia do estado fundamental em hardware sem mitigação de erros.

In [ ]:
# ---- TODO : Tarefa Bônus ---

# Baseado na seção anterior, escolha o melhor backend para executar o código em hardware
best_backend_hw = 

# --- Fim do TODO ---

options = EstimatorOptions(default_shots=100000)
estimator = Estimator(mode=best_backend_hw, options=options)
ground_energy_ext_hw = cost_func_estimator(
    opt_params_sim, circuit_ext_opt_seed, cost_hamiltonian_ext, estimator
)
print(
    f"A energia do estado fundamental do circuito QAOA executado em {best_backend_hw.name} sem EM é: {ground_energy_ext_hw}"
)

Agora aplicamos diferentes técnicas de mitigação e supressão de erros.
As diferentes técnicas que aplicaremos são:

- **Desacoplamento dinâmico**: esta técnica de supressão de erros envolve aplicar uma sequência de pulsos de controle aos qubits ociosos para cancelar interações indesejadas e erros coerentes. Ajuda a preservar a coerência quântica "desacoplando" efetivamente o sistema das fontes de ruído ao longo do tempo.
- **Pauli twirling**: esta técnica de mitigação de erros transforma ruído arbitrário em ruído de Pauli mais simples, aplicando e desfazendo aleatoriamente portas de Pauli em torno das operações, reduzindo o impacto de erros coerentes e permitindo uma modelagem e correção de ruído mais eficazes.
- **TREX**: Twirled Readout Error eXtinction é uma técnica de mitigação de erros de leitura que reduz os erros de medição, invertendo aleatoriamente os qubits antes da medição e corrigindo classicamente os resultados, diagonalizando efetivamente a matriz de erros de leitura e simplificando sua inversão para uma estimativa mais precisa do valor esperado.
- **ZNE**: como explicado anteriormente, ZNE é uma técnica de mitigação de erros que estima o resultado de uma computação quântica como se fosse executada em um dispositivo sem ruído. Para isso, aumenta deliberadamente o ruído de forma controlada, mede os resultados e, em seguida, extrapola de volta para o limite de ruído zero.

In [ ]:
options = EstimatorOptions(default_shots=100000)
# Desacoplamento Dinâmico
options.dynamical_decoupling.enable = True

# Pauli Twirling Probabilístico
options.twirling.enable_gates = True
options.twirling.num_randomizations = 10
options.twirling.shots_per_randomization = 10000

# TREX
options.resilience.measure_mitigation = True
options.resilience.measure_noise_learning.num_randomizations = 10
options.resilience.measure_noise_learning.shots_per_randomization = 10000

# Configuração ZNE
options.resilience.zne_mitigation = True
options.resilience.zne.amplifier = "gate_folding"
options.resilience.zne.extrapolator = "polynomial_degree_2"
options.resilience.zne.noise_factors = (1, 3, 5)

# Executamos em hardware
estimator_EM = Estimator(mode=best_backend_hw, options=options)
ground_energy_ext_hw_EM = cost_func_estimator(
    opt_params_sim, circuit_ext_opt_seed, cost_hamiltonian_ext, estimator_EM
)
print(
    f"A energia do estado fundamental do circuito QAOA executado em {best_backend_hw.name} com EM é: {ground_energy_ext_hw_EM}"
)

Agora podemos usar o primitivo Sampler em hardware real sem nenhuma técnica de mitigação e supressão de erros.

In [ ]:
opt_params_sim = result_qaoa_sim.x
# Executamos em hardware
sampler = Sampler(mode=best_backend_hw)
job = sampler.run([(circuit_ext_opt_seed, opt_params_sim)], shots=SHOTS)
results_sampler = job.result()
counts_list_hw = results_sampler[0].data.meas.get_counts()
display(plot_histogram(counts_list_hw, title=f"Max cut com {best_backend_hw.name} "))

Agora incluímos técnicas de supressão de erros no Sampler.

In [ ]:
opt_params_sim = result_qaoa_sim.x
# Executamos em hardware
sampler = Sampler(mode=best_backend_hw)
# Definir opções de runtime diretamente no sampler
sampler.options.dynamical_decoupling.enable = True
job = sampler.run([(circuit_ext_opt_seed, opt_params_sim)], shots=SHOTS)
results_sampler = job.result()
counts_list_hw_EM = results_sampler[0].data.meas.get_counts()
display(plot_histogram(counts_list_hw_EM, title=f"Max cut com {best_backend_hw.name} com EM"))

### Verificar resultados

In [ ]:
sorted_states_hw = sorted(counts_list_hw.items(), key=lambda item: item[1], reverse=True)
top_states_hw = sorted_states_hw[:num_solutions_ext]
qaoa_ground_states_hw = sorted([state for state, count in top_states_hw])

sorted_states_hw_EM = sorted(counts_list_hw_EM.items(), key=lambda item: item[1], reverse=True)
top_states_hw_EM = sorted_states_hw_EM[:num_solutions_ext]
qaoa_ground_states_hw_EM = sorted([state for state, count in top_states_hw_EM])

print(f"The analytical solutions for the Max-cut problem are: {states_solutions_ext}")
print(
    f"The QAOA ground states solutions for the Max-cut are: {qaoa_ground_states_hw} for {best_backend_hw.name}"
)
print(
    f"The QAOA ground states solutions for the Max-cut are: {qaoa_ground_states_hw_EM} for {best_backend_hw.name} with EM"
)

# Referências

# Referências:

[1] Fahri et al., "A Quantum Approximate Optimization Algorithm" (2014). [arXiv:quant-ph/1411.4028](https://arxiv.org/abs/1411.4028)

[2] Nation et al., "Suppressing Quantum Circuit Errors Due to System Variability" (2023). [PRX Quantum 4, 010327](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.4.010327)

[3] Temme et al., "Error Mitigation for Short-Depth Quantum Circuits" (2017).  [Phys. Rev. Lett. 119, 180509](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.119.180509)

# Informações adicionais

**Criado por:** Jorge Martínez de Lejarza, Alberto Maldonado

**Orientado por:** Marcel Pfaffhauser, Paul Nation, Junye Huang, Sophy Shin

**Versão:** 1.0.1